In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
def generate_polynomial_features(x: np.ndarray, M: int) -> np.ndarray:
    """Generate the polynomial features.

    Args:
        x: A numpy array with shape (N, ).
        M: the degree of the polynomial.
    Returns:
        phi: A feature vector represented by a numpy array with shape (N, M+1);
          each row being (x^{(i)})^j, for 0 <= j <= M.
    """
    N = len(x)
    phi = np.zeros((N, M + 1))
    for m in range(M + 1):
        phi[:, m] = np.power(x, m)
    return phi

In [3]:
n_samples_small = 20
X_small = np.random.rand(n_samples_small)
y_small = 15 * (X_small ** 2) + 25 * X_small + 30 + np.random.randn(n_samples_small) * 10
X_train_small, X_test_small, y_train, y_test = train_test_split(X_small, y_small, test_size=0.2, random_state=42)

X_train = generate_polynomial_features(X_train_small, M=9)
X_test = generate_polynomial_features(X_test_small, M=9)

In [4]:
loaded = np.load('arrays.npz')
X_train = loaded['array1']
X_test = loaded['array2']
y_train = loaded['array3']
y_test = loaded['array4']

## Linear Regression, Lasso and Ridge

In [5]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

model_LR = LinearRegression()
model_LR.fit(X_train, y_train)

y_pred_train = model_LR.predict(X_train)
y_pred_test = model_LR.predict(X_test)

print(f"Linear Regression Model R2 on train data: {r2_score(y_train, y_pred_train)}")
print(f"Linear Regression Model R2 on test data: {r2_score(y_test, y_pred_test)}")

Linear Regression Model R2 on train data: 0.9488675257965271
Linear Regression Model R2 on test data: -0.28887513814431687


In [6]:
print(f"Linear Regression Model intercept: {model_LR.intercept_}")

Linear Regression Model intercept: 110.38487371196155


In [7]:
print(f"Linear Regression Model Coefficients: {model_LR.coef_[1:]}")

Linear Regression Model Coefficients: [-8.10229507e+03  1.62628783e+05 -1.36258544e+06  6.08268540e+06
 -1.59639858e+07  2.54545529e+07 -2.42531081e+07  1.26952881e+07
 -2.80766407e+06]


In [8]:
model_Ridge = Ridge(alpha=0.1)
model_Ridge.fit(X_train, y_train)

y_pred_train = model_Ridge.predict(X_train)
y_pred_test = model_Ridge.predict(X_test)

print(f"Ridge Regression Model R2 on train data: {r2_score(y_train, y_pred_train)}")
print(f"Ridge Regression Model R2 on test data: {r2_score(y_test, y_pred_test)}")

Ridge Regression Model R2 on train data: 0.8173557953167079
Ridge Regression Model R2 on test data: 0.6256731884104297


In [9]:
print(f"Ridge Regression Model intercept: {model_Ridge.intercept_}")

Ridge Regression Model intercept: 31.45350937844156


In [10]:
print(f"Ridge Regression Model Coefficients: {model_Ridge.coef_[1:]}")

Ridge Regression Model Coefficients: [10.93139262 12.95077001  7.50491763  3.50159261  1.46619794  0.75238561
  0.76217181  1.09949949  1.53335468]


In [11]:
model_Lasso = Lasso(alpha=1)
model_Lasso.fit(X_train, y_train)

y_pred_train = model_Lasso.predict(X_train)
y_pred_test = model_Lasso.predict(X_test)

print(f"Lasso Regression Model R2 on train data: {r2_score(y_train, y_pred_train)}")
print(f"Lasso Regression Model R2 on test data: {r2_score(y_test, y_pred_test)}")

Lasso Regression Model R2 on train data: 0.7693665220212011
Lasso Regression Model R2 on test data: 0.5276681116400319


In [12]:
print(f"Lasso Regression Model intercept: {model_Lasso.intercept_}")

Lasso Regression Model intercept: 35.77813122027097


In [13]:
print(f"Lasso Regression Model Coefficients: {model_Lasso.coef_[1:]}")

Lasso Regression Model Coefficients: [ 0.         28.26417145  0.          0.          0.          0.
  0.          0.          0.        ]


## Lasso Regression From Scratch

In [14]:
N = X_train.shape[0]
M = X_train.shape[1]
W = np.ones(M, dtype=float)
X_train_lasso = X_train / (np.linalg.norm(X_train,axis = 0))
X_test_lasso = X_test / (np.linalg.norm(X_train,axis = 0))

In [15]:
def soft_threshold(rho,penalty):
    return np.sign(rho) * max(abs(rho) - penalty, 0.0)

In [16]:
for epoch in range(1000):
    for j in range(M):
        rho = X_train_lasso[:,j].T @ (y_train - (X_train_lasso @ W) + W[j]*X_train_lasso[:,j])
        #z = np.sum(X_train_lasso[:,j]**2)
        if j==0:
            W[j] = rho
        else:
            W[j] = soft_threshold(rho,1)

In [17]:
y_pred_train = X_train_lasso @ W
y_pred_test = X_test_lasso @ W

print(f"Lasso Regression Model R2 on train data: {r2_score(y_train, y_pred_train)}")
print(f"Lasso Regression Model R2 on test data: {r2_score(y_test, y_pred_test)}")

Lasso Regression Model R2 on train data: 0.8227001975450101
Lasso Regression Model R2 on test data: 0.6129552029411226


In [18]:
print(f"Lasso Regression Model intercept: {W[0]}")

Lasso Regression Model intercept: 130.48013880759834


In [19]:
print(f"Lasso Regression Model Coefficients: {W[1:]}")

Lasso Regression Model Coefficients: [ 0.         72.47889223  0.          0.          0.          0.
  0.          0.          3.08352672]


## Ridge Regression From Scratch

In [20]:
reg = 0.1

In [21]:
W = np.linalg.inv(X_train.T @ X_train + reg * np.eye(X_train.shape[1])) @ X_train.T @ y_train

In [22]:
y_pred_train = X_train @ W
y_pred_test = X_test @ W

print(f"Ridge Regression Model R2 on train data: {r2_score(y_train, y_pred_train)}")
print(f"Ridge Regression Model R2 on test data: {r2_score(y_test, y_pred_test)}")

Ridge Regression Model R2 on train data: 0.8176765512165518
Ridge Regression Model R2 on test data: 0.6246445977387474


In [23]:
print(f"Ridge Regression Model intercept: {W[0]}")

Ridge Regression Model intercept: 30.297004975376808


In [24]:
print(f"Ridge Regression Model Coefficients: {W[1:]}")

Ridge Regression Model Coefficients: [13.61860178 12.63928517  6.79545349  2.93484465  1.12959907  0.62077358
  0.78957487  1.24366988  1.76070669]
